In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


def remove_accents(text):
    return unidecode(text)

def get_country_code(country):
    matching_rows = countries_codes.loc[countries_codes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')


leagues_value['Accents Removed'] = leagues_value['Name'].apply(lambda x: unidecode(x))
leagues_value['Team 1 Code'] = leagues_value['Nationality'].apply(get_country_code)

countries_codes['Accents Removed'] = countries_codes['Country'].apply(remove_accents)




/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:55: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_88832/2775728346.py:62: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [2]:
#QUERY GOOGLE FUNCTIONS

#FUNCTIONS

def find_transfermarkt_links(html_content):
    # Regular expression patterns for transfermarkt URLs
    transfermarkt_patterns = [
        r"transfermarkt\.de",
        r"transfermarkt\.us",
        r"transfermarkt\.world"
    ]
    transfermarkt_links = []

    # Find all 'a' tags in the HTML content
    links = html_content.find_all('a', href=True)

    # Filter links that match transfermarkt patterns
    for link in links:
        for pattern in transfermarkt_patterns:
            if re.search(pattern, link['href']):
                transfermarkt_links.append(link['href'])
                break

    return transfermarkt_links


def query_google_1(input_string):
    # Replace spaces with plus signs to format the query string
    query = '+'.join(input_string.split())

    # URL for Google search
    url = f"https://www.google.com/search?q={query}"

    # Send GET request to Google
    response = requests.get(url)

    # Check if request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Return the parsed HTML content
        return soup
    else:
        # If request fails, print error message
        print("Failed to fetch results from Google.")
        return None
    

def extract_name_from_link(link):
        
    name_in_link = link.split('/')[3]
    name_in_link_tokens = name_in_link.split('-')
    capitalized_tokens = []

    for token in name_in_link_tokens:
        capitalized_tokens.append(token.capitalize())

    combined_string = ' '.join(capitalized_tokens)

    return combined_string


def process_google_query(input_text):
    html_result = query_google_1(input_text)
    if html_result:
        transfermarkt_links = find_transfermarkt_links(html_result)
        if transfermarkt_links:
            names_links = []
            for link in transfermarkt_links:
                if link.startswith('/url?q='):
                    link = link.split('/url?q=')[1]
                name_from_link = extract_name_from_link(link)
                names_links.append(name_from_link)
            if len(names_links) == 1:
                name = names_links[0]
                return name
            elif len(names_links) >= 2:
                name_list_matches = list(set(names_links))
                return name_list_matches
            else:
                return 0  # Nobody found
        else:
            return 0  # No links found
    else:
        return 0  # No HTML result

In [3]:
def useSeleniumToFindMoney(
        NAME_TO_FIND, #the name it is looking up
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN):
    if(SALARY_BOOLEAN == False):
        print('finding salary with selenium')
        marketvalue = AssemblyHelpers.seleniumLookUpValueWrapperFunction(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return marketvalue
    elif(SALARY_BOOLEAN == True):
        salary = AssemblyHelpers.capology_selenium_lookup(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return salary

    else:
        return 0

### Adding manual fixes  

#### WC Europe

In [4]:
wcs_europe = load_csv_dataset('world_cup_quals_europe_lineups_complete.csv')

In [7]:
wcs_europe.at[16920, 'Market Value'] = 50000
wcs_europe.at[16922, 'Market Value'] = 50000
wcs_europe.at[16928, 'Market Value'] = 50000

In [10]:
# for index, row in wcs_europe.iterrows():
for i in range(16908, len(wcs_europe)):
    index = i
    row = wcs_europe.iloc[index]
    if row['Status'] != 'SUCCESS':
        name = row["Name(s) Found"]
        nationality = row['Nationality']
        date = row['Date']
        salary_boolean = False
        test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
        if test_figure == 0:
            wcs_europe.at[index, 'Market Value'] = test_figure
            wcs_europe.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            wcs_europe.at[index, 'Lookup Still Required?'] = True
            wcs_europe.at[index, 'Status'] = 'FAIL - ?'
        else:
            wcs_europe.at[index, 'Market Value'] = test_figure
            wcs_europe.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            wcs_europe.at[index, 'Lookup Still Required?'] = False
            wcs_europe.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

In [28]:
wcs_europe.to_csv('world_cup_quals_europe_lineups_complete_EDITED.csv')

#### WC ASIA

In [31]:
wcs_asia = load_csv_dataset('world_cup_qualifiers_asia_lineups_complete.csv')

Status
SUCCESS          13033
FAIL - EMPTY       116
FAIL - NEW          70
FAIL - MANUAL       69
Name: count, dtype: int64

In [50]:
wcs_asia.at[13251, 'Status'] = 'SUCCESS'
wcs_asia.at[13251, 'Market Value'] = 100000
wcs_asia.at[13251, 'Lookup Still Required?'] = False
wcs_asia.at[13251, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13076, 'Status'] = 'SUCCESS'
wcs_asia.at[13076, 'Market Value'] = 0
wcs_asia.at[13076, 'Lookup Still Required?'] = False
wcs_asia.at[13076, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13188, 'Status'] = 'SUCCESS'
wcs_asia.at[13188, 'Market Value'] = 0
wcs_asia.at[13188, 'Lookup Still Required?'] = False
wcs_asia.at[13188, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13190, 'Status'] = 'SUCCESS'
wcs_asia.at[13190, 'Market Value'] = 0
wcs_asia.at[13190, 'Lookup Still Required?'] = False
wcs_asia.at[13190, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13247, 'Status'] = 'SUCCESS'
wcs_asia.at[13247, 'Market Value'] = 0
wcs_asia.at[13247, 'Lookup Still Required?'] = False
wcs_asia.at[13247, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13248, 'Status'] = 'SUCCESS'
wcs_asia.at[13248, 'Market Value'] = 0
wcs_asia.at[13248, 'Lookup Still Required?'] = False
wcs_asia.at[13248, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13264, 'Status'] = 'SUCCESS'
wcs_asia.at[13264, 'Market Value'] = 0
wcs_asia.at[13264, 'Lookup Still Required?'] = False
wcs_asia.at[13264, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13287, 'Status'] = 'SUCCESS'
wcs_asia.at[13287, 'Market Value'] = 0
wcs_asia.at[13287, 'Lookup Still Required?'] = False
wcs_asia.at[13287, 'Lookup Return Case'] = 'Found in Curr Season'

In [107]:
for index, row in wcs_asia.iterrows():
#for i in range(13033, 13288):
    #index = i
    #row = wcs_asia.iloc[index]
    if row['Status'] != 'SUCCESS':
        if row['Status'] == 'FAIL - MANUAL':
            name = row["Name(s) Found"]
            nationality = row['Nationality']
            date = row['Date']
            salary_boolean = False
            test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
            if test_figure == 0:
                wcs_asia.at[index, 'Market Value'] = test_figure
                wcs_asia.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                wcs_asia.at[index, 'Lookup Still Required?'] = True
                wcs_asia.at[index, 'Status'] = 'FAIL - ?'
            else:
                wcs_asia.at[index, 'Market Value'] = test_figure
                wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                wcs_asia.at[index, 'Lookup Still Required?'] = False
                wcs_asia.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is not valid.
Ahmad Salem Mohamad https://www.transfermarkt.us/ahmad-salem-mohamad/nationalmannschaft/spieler/372589
Mohamad Salem https://www.transfermarkt.us/mohamad-salem/nationalmannschaft/spieler/1107627
Mohammed Salem https://www.transfermarkt.us/mohammed-salem/nationalmannschaft/spieler/185640
0 matches []
husain ali pele case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://ww

In [60]:
wcs_asia.at[13033, 'Status'] = 'SUCCESS'
wcs_asia.at[13033, 'Market Value'] = 1000000
wcs_asia.at[13033, 'Lookup Still Required?'] = False
wcs_asia.at[13033, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13037, 'Status'] = 'SUCCESS'
wcs_asia.at[13037, 'Market Value'] = 700000
wcs_asia.at[13037, 'Lookup Still Required?'] = False
wcs_asia.at[13037, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13034, 'Status'] = 'SUCCESS'
wcs_asia.at[13034, 'Market Value'] = 700000
wcs_asia.at[13034, 'Lookup Still Required?'] = False
wcs_asia.at[13034, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13036, 'Status'] = 'SUCCESS'
wcs_asia.at[13036, 'Market Value'] = 700000
wcs_asia.at[13036, 'Lookup Still Required?'] = False
wcs_asia.at[13036, 'Lookup Return Case'] = 'Found in Curr Season'

In [76]:
wcs_asia.at[13058, 'Status'] = 'SUCCESS'
wcs_asia.at[13058, 'Market Value'] = 150000
wcs_asia.at[13058, 'Lookup Still Required?'] = False
wcs_asia.at[13058, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13060, 'Status'] = 'SUCCESS'
wcs_asia.at[13060, 'Market Value'] = 150000
wcs_asia.at[13060, 'Lookup Still Required?'] = False
wcs_asia.at[13060, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13061, 'Status'] = 'SUCCESS'
wcs_asia.at[13061, 'Market Value'] = 150000
wcs_asia.at[13061, 'Lookup Still Required?'] = False
wcs_asia.at[13061, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13062, 'Status'] = 'SUCCESS'
wcs_asia.at[13062, 'Market Value'] = 150000
wcs_asia.at[13062, 'Lookup Still Required?'] = False
wcs_asia.at[13062, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13064, 'Status'] = 'SUCCESS'
wcs_asia.at[13064, 'Market Value'] = 150000
wcs_asia.at[13064, 'Lookup Still Required?'] = False
wcs_asia.at[13064, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13057, 'Status'] = 'SUCCESS'
wcs_asia.at[13057, 'Market Value'] = 75000
wcs_asia.at[13057, 'Lookup Still Required?'] = False
wcs_asia.at[13057, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13066, 'Status'] = 'SUCCESS'
wcs_asia.at[13066, 'Market Value'] = 150000
wcs_asia.at[13066, 'Lookup Still Required?'] = False
wcs_asia.at[13066, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13059, 'Status'] = 'SUCCESS'
wcs_asia.at[13059, 'Market Value'] = 300000
wcs_asia.at[13059, 'Lookup Still Required?'] = False
wcs_asia.at[13059, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13063, 'Status'] = 'SUCCESS'
wcs_asia.at[13063, 'Market Value'] = 300000
wcs_asia.at[13063, 'Lookup Still Required?'] = False
wcs_asia.at[13063, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13065, 'Status'] = 'SUCCESS'
wcs_asia.at[13065, 'Market Value'] = 300000
wcs_asia.at[13065, 'Lookup Still Required?'] = False
wcs_asia.at[13065, 'Lookup Return Case'] = 'Found in Curr Season'
wcs_asia.at[13067, 'Status'] = 'SUCCESS'
wcs_asia.at[13067, 'Market Value'] = 300000
wcs_asia.at[13067, 'Lookup Still Required?'] = False
wcs_asia.at[13067, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13070, 'Status'] = 'SUCCESS'
wcs_asia.at[13070, 'Market Value'] = 50000
wcs_asia.at[13070, 'Lookup Still Required?'] = False
wcs_asia.at[13070, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13073, 'Status'] = 'SUCCESS'
wcs_asia.at[13073, 'Market Value'] = 70000
wcs_asia.at[13073, 'Lookup Still Required?'] = False
wcs_asia.at[13073, 'Lookup Return Case'] = 'Found in Curr Season'

In [83]:
#1 no 2 no 3 no 4 no 5 no 6 no 
for index, row in wcs_asia.iterrows():
    if row['Status'] == 'FAIL - ?':
        wcs_asia.at[index, 'Status'] = 'SUCCESS'
        wcs_asia.at[index, 'Market Value'] = 0
        wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'

In [117]:
for index, row in wcs_asia.iterrows():
    if row['Status'] == 'FAIL - ?':
        if row['Name(s) Found'] == 'Mohanad Salem':
            if row['Season'] == 2016:
                0==0
                wcs_asia.at[index, 'Status'] = 'SUCCESS'
                wcs_asia.at[index, 'Market Value'] = 200000
                wcs_asia.at[index, 'Lookup Still Required?'] = False
                wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            elif row['Season'] == 2017:
                0==0
                wcs_asia.at[index, 'Status'] = 'SUCCESS'
                wcs_asia.at[index, 'Market Value'] = 225000
                wcs_asia.at[index, 'Lookup Still Required?'] = False
                wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            else:
                0==0
                wcs_asia.at[index, 'Status'] = 'SUCCESS'
                wcs_asia.at[index, 'Market Value'] = 25000
                wcs_asia.at[index, 'Lookup Still Required?'] = False
                wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                
            print(index, row['Season'])

In [123]:
wcs_asia.at[13161, 'Status'] = 'SUCCESS'
wcs_asia.at[13161, 'Market Value'] = 300000
wcs_asia.at[13161, 'Lookup Still Required?'] = False
wcs_asia.at[13161, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13246, 'Status'] = 'SUCCESS'
wcs_asia.at[13246, 'Market Value'] = 25000
wcs_asia.at[13246, 'Lookup Still Required?'] = False
wcs_asia.at[13246, 'Lookup Return Case'] = 'Found in Curr Season'

wcs_asia.at[13189, 'Status'] = 'SUCCESS'
wcs_asia.at[13189, 'Market Value'] = 0
wcs_asia.at[13189, 'Lookup Still Required?'] = False
wcs_asia.at[13189, 'Lookup Return Case'] = 'Found in Curr Season'

In [127]:
wcs_asia.to_csv('world_cup_qualifiers_asia_lineups_complete_EDITED.csv')

#### WCQ Africa

In [128]:
wcs_africa = load_csv_dataset('world_cup_qualifiers_africa_lineups_complete.csv')

In [182]:
wcs_africa['Status'].value_counts()

Status
SUCCESS           9373
FAIL - NEW         171
FAIL - NO DATA      66
FAIL - EMPTY         4
Name: count, dtype: int64

In [189]:
for index, row in wcs_africa.iterrows():
    if index >= 9274:
#for i in range(13033, 13288):
    #index = i
    #row = wcs_asia.iloc[index]
        if row['Status'] != 'SUCCESS':
            if row['Status'] == 'FAIL - NEW':
                name = row["Name(s) Found"]
                nationality = row['Nationality']
                date = row['Date']
                salary_boolean = False
                test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
                if test_figure == 0:
                    0==0
                    # wcs_africa.at[index, 'Market Value'] = test_figure
                    # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                    # wcs_africa.at[index, 'Lookup Still Required?'] = True
                    # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
                else:
                    wcs_africa.at[index, 'Market Value'] = test_figure
                    wcs_africa.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    wcs_africa.at[index, 'Lookup Still Required?'] = False
                    wcs_africa.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
        #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/patrick-gerhardt-nyema/marktwertverlauf/spieler/56275
passed 1
passed 2
passed 3
{'value': '2007', 'coordinates': (67.55325610167586, 0.0)} 07
{'value': '2009', 'coordinates': (180.02744348217297, 0.0)} 09
{'value': '2011', 'coordinates': (292.34776740524126, 0.0)} 11
{'value': '2013', 'coordinates': (404.8219547857384, 0.0)} 13
{'value': '2015', 'coordinates': (517.1422787088067, 0.0)} 15
{'value': '2017', 'coordinates': (629.6164660893038, 0.0)} 17
passed 4
passed 5
passed 6
500000.0 12345
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
nayem case
wrapper function nayem case working
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/hugo-marques/ma

In [179]:
for index, row in wcs_africa.iterrows():
    if row['Status'] == 'FAIL - ?':
        if row['Name(s) Found'] == 'Nadir Eltayeb':
            if row['Season'] == 2016:
                0==0
                # wcs_africa.at[index, 'Status'] = 'SUCCESS'
                # wcs_africa.at[index, 'Market Value'] = 0
                # wcs_africa.at[index, 'Lookup Still Required?'] = False
                # wcs_africa.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            elif row['Season'] == 2017:
                0==0
                # wcs_africa.at[index, 'Status'] = 'SUCCESS'
                # wcs_africa.at[index, 'Market Value'] = 0
                # wcs_africa.at[index, 'Lookup Still Required?'] = False
                # wcs_africa.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
            else:
                0==0
                wcs_africa.at[index, 'Status'] = 'SUCCESS'
                wcs_africa.at[index, 'Market Value'] = 0
                wcs_africa.at[index, 'Lookup Still Required?'] = False
                wcs_africa.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                
            print(index, row['Season'])

9163 2013


In [193]:
wcs_africa.to_csv('world_cup_qualifiers_africa_lineups_complete_EDITED.csv')

In [196]:
wcs_asia['Status'].value_counts()

Status
SUCCESS         13119
FAIL - EMPTY      116
FAIL - NEW         53
Name: count, dtype: int64

In [199]:
for index, row in wcs_asia.iterrows():
    if index >= 13227:
#for i in range(13033, 13288):
#index = i
#row = wcs_asia.iloc[index]
        if row['Status'] != 'SUCCESS':
            if row['Status'] == 'FAIL - NEW':
                name = row["Name(s) Found"]
                nationality = row['Nationality']
                date = row['Date']
                salary_boolean = False
                test_figure = useSeleniumToFindMoney(name, nationality, date, salary_boolean)
                if test_figure == 0:
                    0==0
                    # wcs_africa.at[index, 'Market Value'] = test_figure
                    # wcs_africa.at[index, 'Lookup Return Case'] = 'LOOKUP FAIL'
                    # wcs_africa.at[index, 'Lookup Still Required?'] = True
                    # wcs_africa.at[index, 'Status'] = 'FAIL - ?'
                else:
                    wcs_asia.at[index, 'Market Value'] = test_figure
                    wcs_asia.at[index, 'Lookup Return Case'] = 'Found in Curr Season'
                    wcs_asia.at[index, 'Lookup Still Required?'] = False
                    wcs_asia.at[index, 'Status'] = 'SUCCESS'
    else:
        0==0
    #print('were done here')

finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/aloisio/marktwertverlauf/spieler/55246
passed 1
passed 2
passed 3
{'value': '2009', 'coordinates': (120.88471085176799, 0.0)} 09
{'value': '2011', 'coordinates': (200.61123933112697, 0.0)} 11
{'value': '2013', 'coordinates': (280.4469822330604, 0.0)} 13
{'value': '2015', 'coordinates': (360.17351071241944, 0.0)} 15
{'value': '2017', 'coordinates': (440.00925361435293, 0.0)} 17
{'value': '2019', 'coordinates': (519.7357820937119, 0.0)} 19
{'value': '2021', 'coordinates': (599.5715249956454, 0.0)} 21
passed 4
passed 5
passed 6
200000.0 12345
finding salary with selenium
The link is valid.
transfefrmarkt link boolean true hi
https://www.transfermarkt.us/aloisio/marktwertverlauf/spieler/55246
passed 1
passed 2
passed 3
{'value': '2009', 'coordinates': (120.88471085176799, 0.0)} 09
{'value': '2011', 'coordinates': (200.61123933112697, 0.0)} 11
{'value': '2013', 'coordinates': (28